In [1]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
# - 에이전트 간에 전달되는 상태를 정의하는 자료 구조
# - query: 사용자 입력
# - keywords: 추출된 검색 키워드
# - search_results: 검색된 문서 목록 (파일명, 위치, 요약, 관련성 등)
# - context: 검색된 문서들의 결합 내용
# - result: 최종 생성된 답변
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
# - MySQL 데이터베이스 연결을 위한 설정
# - host: localhost
# - user: admin
# - password: 1qazZAQ!
# - db: final
# - charset: utf8mb4 (한글 지원)
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
# - ChromaDB: 벡터 데이터베이스로 문서 임베딩 저장
# - OllamaEmbeddings: 문서 임베딩용 Ollama 모델 (exaone3.5:2.4b)
# - Ollama: 텍스트 생성용 Ollama 모델 (exaone3.5:2.4b)
# - ChatOllama: 챗봇용 Ollama 모델 (exaone3.5:2.4b, temperature=0.1)
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"  # ChromaDB 저장 경로
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 모델
LLM = Ollama(model="exaone3.5:2.4b")  # LLM
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)  # 챗 모델

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
# - RAG 처리 과정을 3D로 시각화하는 클래스
# - 에이전트 상태, 검색 결과, 진행률을 실시간으로 보여줌
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트의 3D 공간 위치 (x, y, z)
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),       # 키워드 추출
            'rag_search': (2.0, 0.0, 1.0),      # 문서 검색
            'answer_generator': (4.0, 0.0, 2.0), # 답변 생성
            'result_formatter': (6.0, 0.0, 1.0)  # 결과 형식화
        }
        self.agent_names = list(self.agent_positions.keys())
        # 에이전트 초기 상태 (대기중)
        self.agent_status = {name: 'waiting' for name in self.agent_names}

        # 3D FigureWidget 생성
        self.fig3d = go.FigureWidget()
        self._init_3d_scene()  # 3D 장면 초기화

        # 관련성 막대 그래프 (검색 결과)
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바 (에이전트 진행 상황)
        self.progress = widgets.FloatProgress(
            value=0.0,
            min=0.0,
            max=100.0,
            description='진행률:',
            bar_style='info'
        )

        # 로그 텍스트 영역 (처리 기록)
        self.log = widgets.Textarea(
            value='',
            placeholder='로그가 여기에 표시됩니다.',
            layout=widgets.Layout(width='100%', height='600px')
        )

        # 전체 UI 레이아웃 (3D + 차트 + 로그)
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        """3D 장면을 초기화하고 에이전트 노드/연결선을 추가"""
        # 에이전트 위치 (x, y, z)
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 에이전트 노드 (마커 + 텍스트)
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers+text',  # 마커와 텍스트 함께 표시
            marker=dict(
                size=14,            # 마커 크기
                color=['gray'] * len(self.agent_names),  # 초기 색상 (회색)
                opacity=0.9,         # 투명도
                line=dict(width=2, color='DarkSlateGrey')  # 마커 테두리
            ),
            text=self.agent_names,   # 노드 텍스트
            textposition="top center",  # 텍스트 위치
            textfont=dict(size=12, color='black'),
            name="에이전트"
        )
        self.fig3d.add_trace(node_trace)

        # 에이전트 간 연결선 (에지)
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i+1]]
            edge = go.Scatter3d(
                x=[a[0], b[0]],
                y=[a[1], b[1]],
                z=[a[2], b[2]],
                mode='lines',
                line=dict(color='lightgray', width=4),
                showlegend=False
            )
            self.fig3d.add_trace(edge)

        # 검색 결과 placeholder (업데이트 예정)
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis',  # 색상 척도
                cmin=0, 
                cmax=100, 
                opacity=0.8, 
                showscale=True,       # 색상 바 표시
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과'
        )
        self.fig3d.add_trace(search_trace)

        # 3D 레이아웃 설정
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
               	xaxis=dict(title='⏩ 진행 단계'),    # 기존: 처리 단계
                yaxis=dict(title='🗂️ 작업 공간'),    # 기존: 데이터 흐름
                zaxis=dict(title='⭐ 중요도(관련성)'), # 기존: 복잡도
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),  # 마진
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)  # 범례
        )

    def show(self):
        """시각화 UI를 노트북에 표시"""
        display(self.container)

    def _append_log(self, text: str):
        """로그 메시지를 추가하고 표시"""
        # 현재 시간 포맷: HH:MM:SS
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{timestamp}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None):
        """
        에이전트 상태를 업데이트하고 시각화를 갱신
        
        Args:
            agent_name: 업데이트할 에이전트 이름
            status: 'processing', 'completed', 'error', 'waiting'
            data: 검색 결과 데이터 (rag_search 에이전트 전용)
        """
        # 1. 로그에 상태 추가
        status_text = status.capitalize()
        self._append_log(f"{agent_name} -> {status_text}")

        # 2. 에이전트 상태 저장
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 3. 색상 매핑: 상태에 따른 색상
        def get_status_color(s):
            if s == 'processing': return 'yellow'
            elif s == 'completed': return 'green'
            elif s == 'error': return 'red'
            else: return 'gray'  # waiting

        # 4. 3D 노드 색상 업데이트
        colors = [get_status_color(self.agent_status[name]) for name in self.agent_names]
        self.fig3d.data[0].marker.color = colors

        # 5. 진행률 업데이트 (완료된 에이전트 비율)
        completed_count = sum(1 for s in self.agent_status.values() if s == 'completed')
        progress_percent = (completed_count / len(self.agent_status)) * 100
        self.progress.value = progress_percent

        # 6. 검색 결과가 있는 경우 시각화 업데이트
        if data and agent_name == 'rag_search':
            self.update_search_results(data)

    def update_search_results(self, results: List[Dict[str, Any]]):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트
        
        Args:
            results: 검색 결과 목록 (relevance, file_name, ...)
        """
        # 1. 관련성 순으로 내림차순 정렬
        results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
        
        # 2. 3D 검색 결과 업데이트
        base_x, base_y, base_z = self.agent_positions['rag_search']  # rag_search 위치
        n = len(results_sorted)
        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        radius = 1.5  # 원형 배치 반지름

        # 3. 검색 결과를 원형으로 배치
        for i, res in enumerate(results_sorted):
            angle = (i * 2 * np.pi) / n  # 각도
            # 3D 좌표: rag_search 위치를 중심으로 원형 배치
            x = base_x + radius * np.cos(angle)
            y = base_y + radius * np.sin(angle)
            z = base_z + (res.get('relevance', 0) / 100) * 2  # 관련성에 따른 높이
            
            xs.append(x)
            ys.append(y)
            zs.append(z)
            sizes.append(max(6, res.get('relevance', 0) / 4))  # 관련성에 따른 크기
            colors.append(res.get('relevance', 0))  # 색상 (관련성)
            # 텍스트: 파일명 + 관련성
            text = f"{res.get('file_name', '파일')}<br>{res.get('relevance', 0)}%"
            texts.append(text)

        # 4. 3D 검색 결과 trace 업데이트
        for trace in self.fig3d.data:
            if trace.name == '검색 결과':
                trace.x = xs
                trace.y = ys
                trace.z = zs
                trace.marker.size = sizes
                trace.marker.color = colors
                trace.text = texts
                break

        # 5. 막대 차트 업데이트 (관련성)
        file_names = [res.get('file_name', '') for res in results_sorted]
        relevances = [res.get('relevance', 0) for res in results_sorted]

        # FigureWidget의 실시간 업데이트를 위해 batch_update 사용 권장
        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = file_names
            self.bar_fig.data[0].y = relevances
            self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
            
            # 핵심 수정: 색상을 데이터 값(relevances)에 매핑
            self.bar_fig.data[0].marker.color = relevances
            
            # 'Viridis' 스케일: 낮은 값(보라색/어두움) -> 높은 값(노란색/밝음)
            self.bar_fig.data[0].marker.colorscale = 'Viridis'
            
            # 색상 범위 고정 (0~100%) - 선택 사항이지만 일관된 색상 표현에 도움
            self.bar_fig.data[0].marker.cmin = 0
            self.bar_fig.data[0].marker.cmax = 100
            self.bar_fig.data[0].marker.showscale = True # 우측에 색상바 표시

        # 6. 막대 차트 레이아웃 업데이트 (너비 조절)
        self.bar_fig.update_layout(
            xaxis_tickangle=-45,  # 파일명 45도 회전
            margin=dict(b=100)    # 파일명 간격 확보
        )


# ================================================================
# 5. 시각화 객체 생성
# ================================================================
# - 위에서 정의한 RAGNotebookVisualizer 인스턴스 생성
# - 노트북에서 show() 메소드를 호출해 UI 표시
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================
# - 4개의 에이전트: 키워드 추출, RAG 검색, 답변 생성, 결과 형식화
# - 각 에이전트는 visualizer.update_agent_status()로 상태를 업데이트
# ================================================================

# 6.1. 키워드 추출 에이전트
def extractor_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('extractor', 'processing')
    
    # 프롬프트 템플릿: 사용자 질의에서 키워드 추출
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    # 프롬프트 포맷팅
    formatted_prompt = keyword_prompt.format(query=state["query"])
    # LLM 호출
    keywords = LLM.invoke(formatted_prompt)
    
    # 상태: completed
    visualizer.update_agent_status('extractor', 'completed')
    # state 업데이트 (keywords 추가)
    return {**state, "keywords": keywords.strip()}

# 6.2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        # 1. ChromaDB 로드
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        # 2. 벡터스토어 검색 (k=10: 상위 10개)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            # 검색 결과 없음
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 3. 문서 중복 제거 (최고 유사도 유지)
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")  # 문서 ID
            if doc_id:
                content = doc.page_content.strip()
                if not content:  # 내용이 빈 경우 스킵
                    continue
                # 동일 문서 ID 중 가장 높은 유사도를 유지
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 4. 유사도 정규화 (0-100%로 변환)
        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        # 5. MySQL에서 파일 메타데이터 조회
        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                # MySQL: documents 테이블에서 문서 ID로 조회
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue  # 데이터 없을 경우 스킵

                # 6. 관련성 % 계산
                if min_score == max_score:
                    relevance = 100.0  # 모든 문서 동일 유사도
                else:
                    # min-max normalization: 0-100%로 변환
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                # 7. 검색 결과 목록에 추가
                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            # 8. 관련성 순으로 정렬 (내림차순) 및 상위 10개
            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            # 9. 문서 내용 결합 (context)
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            # MySQL 오류 처리
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        # 상태: completed + 검색 결과 데이터 전달
        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        # 일반 오류 처리
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

# 6.3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('answer_generator', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        # 1. 프롬프트 템플릿
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        # 2. 검색 결과 요약 생성
        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        # 3. LLM 체인: 프롬프트 + LLM + 출력 파서
        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        # 상태: completed
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 6.4. 결과 형식화 에이전트
def result_formatter_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('result_formatter', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        # 1. 검색 결과 형식화 (Markdown 스타일)
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        # 상태: completed
        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
# - 에이전트들을 노드로 연결한 그래프 생성
# - 실행 순서: extractor → rag_search → answer_generator → result_formatter
# ================================================================
graph = StateGraph(AgentState)  # AgentState를 상태로 사용하는 그래프
graph.add_node("extractor", extractor_agent)  # 노드 추가
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

# 시작 노드 설정
graph.set_entry_point("extractor")

# 노드 연결
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)  # END: 그래프 종료

# 그래프 컴파일
app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
# 1) 시각화 UI 표시
visualizer.show()

# 2) RAG 파이프라인 실행
state = {
    "query": "한강에 버스가 있는 내용",  # 사용자 질의
    "keywords": "",            # 초기 키워드
    "search_results": [],      # 검색 결과
    "context": "",             # 문서 내용
    "result": ""               # 최종 답변
}

# 3) invoke()로 그래프 실행
result = app.invoke(state)

# 4) 최종 결과 출력
print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

# (visualizer는 에이전트 실행 도중 실시간으로 업데이트됩니다.)

/tmp/ipykernel_3520978/131628866.py:73: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 모델
/tmp/ipykernel_3520978/131628866.py:74: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  LLM = Ollama(model="exaone3.5:2.4b")  # LLM
/tmp/ipykernel_3520978/131628866.py:75: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.

    'data': [{'marker': {'color': [gray, gray, gray, gray],
     …

/tmp/ipykernel_3520978/131628866.py:366: LangChainDeprecationWarning:

The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.




📊 RAG 처리 완료! 최종 결과:

🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: IT과학정보.docx
📁 위치: fileList/IT과학정보.docx
📝 요약: SK텔레콤이 SK AX와 협력하여 개발한 AI 업무 지원 시스템 '에이닷 비즈'가 연말까지 SK그룹 내 25개 사에 도입될 예정이다. 이 시스템은 자연어 처리를 통해 회의록 작성 시간을 60%, 보고서 작성 시간을 40% 단축시키며, 정보 검색, 일정 관리, 채용 등 다양한 업무를 지원한다. 에이전트 빌더와 스토어 기능을 통해 IT 지식이 없는 구성원도 쉽...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (99.4%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (90.2%) ---
📄 파일명: 해외축구_기사.docx
📁 위치: fileList/해외축구_기사.docx
📝 요약: 아틀레티코 마드리드는 잉글랜드 출신 공격수 메이슨 그린우드를 영입하려는 의지를 보이며, 마르세유와는 이적료 약 1225억원으로 협상 중이다. 그린우드는 맨체스터 유나이티드 유스 시절부터 두각을 나타내다가 2021년 이후 법적 문제로 인해 위기를 겪었으나, 이후 마르세유로 이적해 리그앙 득점왕에 오르며 다시 주목받았다. 현재 그린우드의 기세는 이어지고 있어,...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (82.7%) ---
📄 파일명: IT기사.docx
📁 위치: fi

# 코드 수정

In [4]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
# - 에이전트 간에 전달되는 상태를 정의하는 자료 구조
# - query: 사용자 입력
# - keywords: 추출된 검색 키워드
# - search_results: 검색된 문서 목록 (파일명, 위치, 요약, 관련성 등)
# - context: 검색된 문서들의 결합 내용
# - result: 최종 생성된 답변
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
# - MySQL 데이터베이스 연결을 위한 설정
# - host: localhost
# - user: admin
# - password: 1qazZAQ!
# - db: final
# - charset: utf8mb4 (한글 지원)
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
# - ChromaDB: 벡터 데이터베이스로 문서 임베딩 저장
# - OllamaEmbeddings: 문서 임베딩용 Ollama 모델 (exaone3.5:2.4b)
# - Ollama: 텍스트 생성용 Ollama 모델 (exaone3.5:2.4b)
# - ChatOllama: 챗봇용 Ollama 모델 (exaone3.5:2.4b, temperature=0.1)
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"  # ChromaDB 저장 경로
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 모델
LLM = Ollama(model="exaone3.5:2.4b")  # LLM
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)  # 챗 모델

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
# - RAG 처리 과정을 3D로 시각화하는 클래스
# - 에이전트 상태, 검색 결과, 진행률을 실시간으로 보여줌
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트의 3D 공간 위치 (x, y, z)
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),       # 키워드 추출
            'rag_search': (2.0, 0.0, 1.0),      # 문서 검색
            'answer_generator': (4.0, 0.0, 2.0), # 답변 생성
            'result_formatter': (6.0, 0.0, 1.0)  # 결과 형식화
        }
        self.agent_names = list(self.agent_positions.keys())
        # 에이전트 초기 상태 (대기중)
        self.agent_status = {name: 'waiting' for name in self.agent_names}

        # 3D FigureWidget 생성
        self.fig3d = go.FigureWidget()
        self._init_3d_scene()  # 3D 장면 초기화

        # 관련성 막대 그래프 (검색 결과)
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바 (에이전트 진행 상황)
        self.progress = widgets.FloatProgress(
            value=0.0,
            min=0.0,
            max=100.0,
            description='진행률:',
            bar_style='info'
        )

        # 로그 텍스트 영역 (처리 기록)
        self.log = widgets.Textarea(
            value='',
            placeholder='로그가 여기에 표시됩니다.',
            layout=widgets.Layout(width='100%', height='600px')
        )

        # 전체 UI 레이아웃 (3D + 차트 + 로그)
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        """3D 장면을 초기화하고 에이전트 노드/연결선을 추가"""
        # 에이전트 위치 (x, y, z)
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 🆕 1. 에이전트 노드 그리기 (삭제 또는 투명화)
        # 에이전트 노드를 그리지 않거나, 렌더링만 해놓고 투명하게 처리하여 숨깁니다.
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers+text',
            marker=dict(
                size=14,
                color=['rgba(0,0,0,0)'],  # 완전히 투명하게 처리
                opacity=0.0,
                line=dict(width=0, color='rgba(0,0,0,0)')
            ),
            text=[''] * len(self.agent_names),  # 텍스트도 비움
            textposition="top center",
            name="에이전트",
            showlegend=False # 범례에서도 숨김
        )
        self.fig3d.add_trace(node_trace)


        # 검색 결과 placeholder (업데이트 예정)
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis',
                cmin=0, 
                cmax=100, 
                opacity=0.8, 
                showscale=True,
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과'
        )
        self.fig3d.add_trace(search_trace)

        # 3. 3D 레이아웃 설정
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
                xaxis=dict(title=''),    # 축 레이블 삭제
                yaxis=dict(title=''),    # 축 레이블 삭제
                zaxis=dict(title='⭐ 중요도(관련성)'), 
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),  # 마진
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)  # 범례
        )

    def show(self):
        """시각화 UI를 노트북에 표시"""
        display(self.container)

    def _append_log(self, text: str):
        """로그 메시지를 추가하고 표시"""
        # 현재 시간 포맷: HH:MM:SS
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{timestamp}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None):
        """
        에이전트 상태를 업데이트하고 시각화를 갱신
        
        Args:
            agent_name: 업데이트할 에이전트 이름
            status: 'processing', 'completed', 'error', 'waiting'
            data: 검색 결과 데이터 (rag_search 에이전트 전용)
        """
        # 1. 로그에 상태 추가
        status_text = status.capitalize()
        self._append_log(f"{agent_name} -> {status_text}")

        # 2. 에이전트 상태 저장
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 3. 색상 매핑: 상태에 따른 색상
        def get_status_color(s):
            if s == 'processing': return 'yellow'
            elif s == 'completed': return 'green'
            elif s == 'error': return 'red'
            else: return 'gray'  # waiting

        # 4. 3D 노드 색상 업데이트
        colors = [get_status_color(self.agent_status[name]) for name in self.agent_names]
        self.fig3d.data[0].marker.color = colors

        # 5. 진행률 업데이트 (완료된 에이전트 비율)
        completed_count = sum(1 for s in self.agent_status.values() if s == 'completed')
        progress_percent = (completed_count / len(self.agent_status)) * 100
        self.progress.value = progress_percent

        # 6. 검색 결과가 있는 경우 시각화 업데이트
        if data and agent_name == 'rag_search':
            self.update_search_results(data)

    def update_search_results(self, results: List[Dict[str, Any]]):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트
        
        Args:
            results: 검색 결과 목록 (relevance, file_name, ...)
        """
        # 1. 관련성 순으로 내림차순 정렬
        results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
        
        # 2. 3D 검색 결과 업데이트
        base_x, base_y, base_z = self.agent_positions['rag_search']  # rag_search 위치
        n = len(results_sorted)
        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        radius = 1.5  # 원형 배치 반지름

        # 3. 검색 결과를 원형으로 배치
        for i, res in enumerate(results_sorted):
            angle = (i * 2 * np.pi) / n  # 각도
            # 3D 좌표: rag_search 위치를 중심으로 원형 배치
            x = base_x + radius * np.cos(angle)
            y = base_y + radius * np.sin(angle)
            z = base_z + (res.get('relevance', 0) / 100) * 2  # 관련성에 따른 높이
            
            xs.append(x)
            ys.append(y)
            zs.append(z)
            sizes.append(max(6, res.get('relevance', 0) / 4))  # 관련성에 따른 크기
            colors.append(res.get('relevance', 0))  # 색상 (관련성)
            # 텍스트: 파일명 + 관련성
            text = f"{res.get('file_name', '파일')}<br>{res.get('relevance', 0)}%"
            texts.append(text)

        # 4. 3D 검색 결과 trace 업데이트
        for trace in self.fig3d.data:
            if trace.name == '검색 결과':
                trace.x = xs
                trace.y = ys
                trace.z = zs
                trace.marker.size = sizes
                trace.marker.color = colors
                trace.text = texts
                break

        # 5. 막대 차트 업데이트 (관련성)
        file_names = [res.get('file_name', '') for res in results_sorted]
        relevances = [res.get('relevance', 0) for res in results_sorted]

        # FigureWidget의 실시간 업데이트를 위해 batch_update 사용 권장
        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = file_names
            self.bar_fig.data[0].y = relevances
            self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
            
            # 핵심 수정: 색상을 데이터 값(relevances)에 매핑
            self.bar_fig.data[0].marker.color = relevances
            
            # 'Viridis' 스케일: 낮은 값(보라색/어두움) -> 높은 값(노란색/밝음)
            self.bar_fig.data[0].marker.colorscale = 'Viridis'
            
            # 색상 범위 고정 (0~100%) - 선택 사항이지만 일관된 색상 표현에 도움
            self.bar_fig.data[0].marker.cmin = 0
            self.bar_fig.data[0].marker.cmax = 100
            self.bar_fig.data[0].marker.showscale = True # 우측에 색상바 표시

        # 6. 막대 차트 레이아웃 업데이트 (너비 조절)
        self.bar_fig.update_layout(
            xaxis_tickangle=-45,  # 파일명 45도 회전
            margin=dict(b=100)    # 파일명 간격 확보
        )


# ================================================================
# 5. 시각화 객체 생성
# ================================================================
# - 위에서 정의한 RAGNotebookVisualizer 인스턴스 생성
# - 노트북에서 show() 메소드를 호출해 UI 표시
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================
# - 4개의 에이전트: 키워드 추출, RAG 검색, 답변 생성, 결과 형식화
# - 각 에이전트는 visualizer.update_agent_status()로 상태를 업데이트
# ================================================================

# 6.1. 키워드 추출 에이전트
def extractor_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('extractor', 'processing')
    
    # 프롬프트 템플릿: 사용자 질의에서 키워드 추출
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    # 프롬프트 포맷팅
    formatted_prompt = keyword_prompt.format(query=state["query"])
    # LLM 호출
    keywords = LLM.invoke(formatted_prompt)
    
    # 상태: completed
    visualizer.update_agent_status('extractor', 'completed')
    # state 업데이트 (keywords 추가)
    return {**state, "keywords": keywords.strip()}

# 6.2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        # 1. ChromaDB 로드
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        # 2. 벡터스토어 검색 (k=10: 상위 10개)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            # 검색 결과 없음
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 3. 문서 중복 제거 (최고 유사도 유지)
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")  # 문서 ID
            if doc_id:
                content = doc.page_content.strip()
                if not content:  # 내용이 빈 경우 스킵
                    continue
                # 동일 문서 ID 중 가장 높은 유사도를 유지
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 4. 유사도 정규화 (0-100%로 변환)
        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        # 5. MySQL에서 파일 메타데이터 조회
        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                # MySQL: documents 테이블에서 문서 ID로 조회
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue  # 데이터 없을 경우 스킵

                # 6. 관련성 % 계산
                if min_score == max_score:
                    relevance = 100.0  # 모든 문서 동일 유사도
                else:
                    # min-max normalization: 0-100%로 변환
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                # 7. 검색 결과 목록에 추가
                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            # 8. 관련성 순으로 정렬 (내림차순) 및 상위 10개
            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            # 9. 문서 내용 결합 (context)
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            # MySQL 오류 처리
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        # 상태: completed + 검색 결과 데이터 전달
        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        # 일반 오류 처리
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

# 6.3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('answer_generator', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        # 1. 프롬프트 템플릿
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        # 2. 검색 결과 요약 생성
        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        # 3. LLM 체인: 프롬프트 + LLM + 출력 파서
        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        # 상태: completed
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 6.4. 결과 형식화 에이전트
def result_formatter_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('result_formatter', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        # 1. 검색 결과 형식화 (Markdown 스타일)
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        # 상태: completed
        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
# - 에이전트들을 노드로 연결한 그래프 생성
# - 실행 순서: extractor → rag_search → answer_generator → result_formatter
# ================================================================
graph = StateGraph(AgentState)  # AgentState를 상태로 사용하는 그래프
graph.add_node("extractor", extractor_agent)  # 노드 추가
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

# 시작 노드 설정
graph.set_entry_point("extractor")

# 노드 연결
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)  # END: 그래프 종료

# 그래프 컴파일
app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
# 1) 시각화 UI 표시
visualizer.show()

# 2) RAG 파이프라인 실행
state = {
    "query": "한강에 버스가 있는 내용",  # 사용자 질의
    "keywords": "",            # 초기 키워드
    "search_results": [],      # 검색 결과
    "context": "",             # 문서 내용
    "result": ""               # 최종 답변
}

# 3) invoke()로 그래프 실행
result = app.invoke(state)

# 4) 최종 결과 출력
print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

# (visualizer는 에이전트 실행 도중 실시간으로 업데이트됩니다.)

    'data': [{'marker': {'color': [rgba(0,0,0,0)],
              …


📊 RAG 처리 완료! 최종 결과:

🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (64.1%) ---
📄 파일명: 파렴치한.docx
📁 위치: fileList/파렴치한.docx
📝 요약: 지난달 20일, 유튜브 채널 '한문철 TV'에 제보된 사연에서, 주차 자리를 맡고 있던 아줌마와의 충돌로 인해 차주 A씨는 특수폭행으로 고소당했다. A씨는 아주머니가 자신의 차와 약간 접촉한 후에도 이를 무시하고 주차를 완료했으나, 5분 뒤 아주머니의 남편으로부터 고의적 폭행 주장에 대한 전화를 받고 고소를 당했다. A씨는 주차 시 주변 상황 확인 부족을 ...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (51.5%) ---
📄 파일명: IT기사.docx
📁 위치: fileList/IT기사.docx
📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (47.0%) ---
📄 파일명: 2025년 신문 구독 지원 신청 전 필독사항.pdf

# 진행률 삭제

In [11]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),
            'rag_search': (2.0, 0.0, 1.0),
            'answer_generator': (4.0, 0.0, 2.0),
            'result_formatter': (6.0, 0.0, 1.0)
        }
        self.agent_names = list(self.agent_positions.keys())
        self.agent_status = {name: 'waiting' for name in self.agent_names}

        self.fig3d = go.FigureWidget()
        self._init_3d_scene()

        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바와 로그 제거, 3D + 차트만 유지
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 에이전트 노드를 투명하게 처리하여 시각화에서 제거
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers+text',
            marker=dict(
                size=14,
                color=['rgba(0,0,0,0)'],
                opacity=0.0,
                line=dict(width=0, color='rgba(0,0,0,0)')
            ),
            text=[''] * len(self.agent_names),
            textposition="top center",
            name="에이전트",
            showlegend=False
        )
        self.fig3d.add_trace(node_trace)

        # 검색 결과 placeholder
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis',
                cmin=0, 
                cmax=100, 
                opacity=0.8, 
                showscale=True,
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과'
        )
        self.fig3d.add_trace(search_trace)

        # 3D 레이아웃 설정 - X/Y 축 레이블 삭제
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
                xaxis=dict(title=''),    # 진행단계 레이블 삭제
                yaxis=dict(title=''),    # 작업 공간 레이블 삭제
                zaxis=dict(title='⭐ 중요도(관련성)'),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),
        )

    def show(self):
        display(self.container)

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None):
        """에이전트 상태 업데이트 (진행률 제거)"""
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 검색 결과가 있는 경우 시각화 업데이트
        if data and agent_name == 'rag_search':
            self.update_search_results(data)

    def update_search_results(self, results: List[Dict[str, Any]]):
        results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
        
        base_x, base_y, base_z = self.agent_positions['rag_search']
        n = len(results_sorted)
        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        radius = 1.5

        for i, res in enumerate(results_sorted):
            angle = (i * 2 * np.pi) / n
            x = base_x + radius * np.cos(angle)
            y = base_y + radius * np.sin(angle)
            z = base_z + (res.get('relevance', 0) / 100) * 2
            
            xs.append(x)
            ys.append(y)
            zs.append(z)
            sizes.append(max(6, res.get('relevance', 0) / 4))
            colors.append(res.get('relevance', 0))
            text = f"{res.get('file_name', '파일')}<br>{res.get('relevance', 0)}%"
            texts.append(text)

        for trace in self.fig3d.data:
            if trace.name == '검색 결과':
                trace.x = xs
                trace.y = ys
                trace.z = zs
                trace.marker.size = sizes
                trace.marker.color = colors
                trace.text = texts
                break

        file_names = [res.get('file_name', '') for res in results_sorted]
        relevances = [res.get('relevance', 0) for res in results_sorted]

        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = file_names
            self.bar_fig.data[0].y = relevances
            self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
            self.bar_fig.data[0].marker.color = relevances
            self.bar_fig.data[0].marker.colorscale = 'Viridis'
            self.bar_fig.data[0].marker.cmin = 0
            self.bar_fig.data[0].marker.cmax = 100
            self.bar_fig.data[0].marker.showscale = True

        self.bar_fig.update_layout(
            xaxis_tickangle=-45,
            margin=dict(b=100)
        )

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================

def extractor_agent(state: AgentState):
    visualizer.update_agent_status('extractor', 'processing')
    
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    formatted_prompt = keyword_prompt.format(query=state["query"])
    keywords = LLM.invoke(formatted_prompt)
    
    visualizer.update_agent_status('extractor', 'completed')
    return {**state, "keywords": keywords.strip()}

def rag_search_agent(state: AgentState):
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")
            if doc_id:
                content = doc.page_content.strip()
                if not content:
                    continue
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue

                if min_score == max_score:
                    relevance = 100.0
                else:
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

def answer_generator_agent(state: AgentState):
    visualizer.update_agent_status('answer_generator', 'processing')
    
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

def result_formatter_agent(state: AgentState):
    visualizer.update_agent_status('result_formatter', 'processing')
    
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)

app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
visualizer.show()

state = {
    "query": "한강에 버스가 있는 내용",
    "keywords": "",
    "search_results": [],
    "context": "",
    "result": ""
}

result = app.invoke(state)

print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

    'data': [{'marker': {'color': [rgba(0,0,0,0)],
              …


📊 RAG 처리 완료! 최종 결과:

🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (61.2%) ---
📄 파일명: 파렴치한.docx
📁 위치: fileList/파렴치한.docx
📝 요약: 지난달 20일, 유튜브 채널 '한문철 TV'에 제보된 사연에서, 주차 자리를 맡고 있던 아줌마와의 충돌로 인해 차주 A씨는 특수폭행으로 고소당했다. A씨는 아주머니가 자신의 차와 약간 접촉한 후에도 이를 무시하고 주차를 완료했으나, 5분 뒤 아주머니의 남편으로부터 고의적 폭행 주장에 대한 전화를 받고 고소를 당했다. A씨는 주차 시 주변 상황 확인 부족을 ...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (50.5%) ---
📄 파일명: IT기사.docx
📁 위치: fileList/IT기사.docx
📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (45.1%) ---
📄 파일명: 2025년 신문 구독 지원 신청 전 필독사항.pdf

# 질문 기준 유사도 기준으로 시각화

    'data': [{'marker': {'color': 'red', 'line': {'color': 'darkr…


📊 RAG 처리 완료! 최종 결과:
🔍 검색 결과:

1. 속도미달.docx (100.0%)
   📁 위치: fileList/속도미달.docx
   📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된...
   🏷️ 유형: .docx
2. IT기사.docx (79.5%)
   📁 위치: fileList/IT기사.docx
   📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐...
   🏷️ 유형: .docx
3. IT과학정보.docx (65.8%)
   📁 위치: fileList/IT과학정보.docx
   📝 요약: SK텔레콤이 SK AX와 협력하여 개발한 AI 업무 지원 시스템 '에이닷 비즈'가 연말까지 SK그룹 내 25개 사에 도입될 예정이다. 이 시스템은 자연어 처리를 통해 회의록 작성 ...
   🏷️ 유형: .docx
4. 세계기사.docx (65.4%)
   📁 위치: fileList/세계기사.docx
   📝 요약: 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 과도한 가족 압박과 외로움으로 인해 뉴질랜드 이주 후 다양한 도시에서 생활하며 학문적 성공을 이루었으나, 현재는 한 달에 약 2...
   🏷️ 유형: .docx
5. 파렴치한.docx (47.2%)
   📁 위치: fileList/파렴치한.docx
   📝 요약: 지난달 20일, 유튜브 채널 '한문철 TV'에 제보된 사연에서, 주차 자리를 맡고 있던 아줌마와의 충돌로 인해 차주 A씨는 특수폭행으로 고소당했다. A씨는 아주머니가 자신의 차와 ...
   🏷️ 유형: .docx
6. 2025년 신문 구독 지원 신청 전 필독사항.pdf (46.2%)
   📁 위치: fileList/2025년 신문 구독 지원 신

# 관련성 구조

In [44]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드 (질문 중심)
# ================================================================
# - 사용자가 입력한 질의(Query)를 3D 공간의 중앙(0,0,0)에 배치.
# - 검색된 문서들은 관련성(유사도)이 높을수록 중앙의 질문에 가깝게 표시.
# - LangGraph 에이전트들의 처리 과정 시각화 요소는 제거.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langgraph.graph import StateGraph, END # LangGraph 컴포넌트는 여전히 로직 제어

import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display
# datetime은 로그 제거로 인해 더 이상 필요하지 않음

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# - 에이전트 처리 과정 모니터링 기능 제거, 질문 중심 시각화로 전환
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 질문(쿼리) 노드의 3D 공간 위치를 중앙으로 고정
        self.query_position = (0.0, 0.0, 0.0)

        self.fig3d = go.FigureWidget()
        self._init_scene()  # 3D 장면 초기화

        # 관련성 막대 그래프 (검색 결과)
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바 및 로그 위젯 제거
        # UI 레이아웃 구성: 3D 그래프와 막대 그래프만 표시
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_scene(self):
        """3D 장면 초기화: 질문 중심 배치"""
        # 1. 쿼리 노드 (중앙, 빨간색 다이아몬드)
        self.query_trace_idx = 0 # 가장 먼저 추가되는 trace
        self.fig3d.add_trace(go.Scatter3d(
            x=[0], y=[0], z=[0], # 중앙 고정
            mode='markers+text',
            marker=dict(
                size=5, # 이전보다 약간 크게
                color='red',
                symbol='diamond',
                line=dict(width=2, color='darkred')
            ),
            text=['❓ Query'], # 초기 텍스트
            textposition='bottom center',
            name='사용자 질문',
            showlegend=True
        ))
        
        # 2. 검색 결과 노드 (초기 empty, update_search_results에서 채움)
        self.search_trace_idx = 1
        self.fig3d.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis', # 관련성 높을수록 밝은 노란색
                cmin=0, cmax=100, 
                opacity=0.9, 
                showscale=True,
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과'
        ))
        
        # 3. 쿼리-결과 연결선 (빨간 점선)
        self.query_edge_trace_idx = 2
        self.fig3d.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode='lines',
            line=dict(color='rgba(255,0,0,0.7)', width=2, dash='dot'),
            name='쿼리-문서 연결',
            showlegend=False # 범례에 나타내지 않음
        ))
        
        # 4. 레이아웃 설정 (X, Y 축 숨김, Z 축은 거리 의미)
        self.fig3d.update_layout(
            title='🔍 RAG: 질문 기반 문서 검색 (가까울수록 관련성 높음)',
            scene=dict(
                xaxis=dict(visible=False, showticklabels=False, showgrid=False, zeroline=False),
                yaxis=dict(visible=False, showticklabels=False, showgrid=False, zeroline=False),
                zaxis=dict(title='거리 (관련성 ↓)', showgrid=True), # Z축이 거리를 나타냄
                camera=dict(eye=dict(x=1.1, y=1.1, z=0.8)) # 초기 카메라 시점 조정
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )

    def show(self):
        """시각화 UI를 노트북에 표시"""
        display(self.container)

    def _update_query_text(self, query: str):
        """중앙 쿼리 노드의 텍스트 업데이트"""
        q_text = f"❓ {query[:30]}" + ("..." if len(query) > 30 else "")
        self.fig3d.data[self.query_trace_idx].text = [q_text]

    def update_search_results(self, results: List[Dict[str, Any]], current_query: str):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트"""
        if not results: return # 결과 없으면 업데이트 안 함
        
        # 쿼리 노드 텍스트를 먼저 업데이트
        self._update_query_text(current_query)

        # 1. 관련성 순으로 내림차순 정렬
        res_sorted = sorted(results, key=lambda x: x.get("relevance", 0), reverse=True)
        n = len(res_sorted)
        
        bx, by, bz = self.query_position # 쿼리 노드 중앙 좌표

        xs, ys, zs = [], [], []
        edge_xs, edge_ys, edge_zs = [], [], []
        sizes, colors, texts = [], [], []

        # 배치 파라미터: 유사도가 높을수록 쿼리에 가까움
        min_dist = 0.5  # 관련성 100%일 때의 최소 거리
        max_dist = 4.0  # 관련성 0%일 때의 최대 거리
        
        # Golden angle spiral 배치로 구형 표면에 고르게 분포
        golden_angle = np.pi * (3 - np.sqrt(5)) # 약 2.39996 라디안

        for i, item in enumerate(res_sorted):
            rel = float(item.get("relevance", 0.0))
            
            # 거리를 관련성에 반비례하게 설정 (rel 높으면 dist 작아짐)
            dist = min_dist + (1 - rel / 100) * (max_dist - min_dist)
            
            # 구면 좌표 계산
            # 원점(쿼리)에서 멀어질수록 z값이 커지도록 (z축이 거리의 의미를 갖도록)
            # 여기서는 z축을 거리로 사용하므로, x, y는 평면 분포에 사용
            theta = i * golden_angle # 0 ~ 2pi
            # 문서가 많을 때 서로 겹치지 않게 z축에도 분산
            # 간단하게 z축을 높이로 사용하여 dist를 표현할 수도 있습니다.
            # dist 자체를 z축으로 사용하고, x, y는 랜덤 또는 원형으로 분산
            
            # 현재 구현은 구면 좌표계에서 (x,y,z)를 거리 dist로 만드는 방식입니다.
            # 즉, dist가 작으면 x,y,z 모두 0에 가깝게 됩니다.
            # (phi는 0에서 pi까지 분포하여 구의 위아래를 나타냅니다.)
            phi = np.arccos(1 - 2 * (i + 0.5) / max(n, 1)) # 0 ~ pi
            
            x = bx + dist * np.sin(phi) * np.cos(theta)
            y = by + dist * np.sin(phi) * np.sin(theta)
            z = bz + dist * np.cos(phi) # 이 z값이 쿼리로부터의 높이를 나타냅니다.
            
            xs.append(x); ys.append(y); zs.append(z)
            
            # 쿼리(중앙)에서 각 문서 노드로 연결선 추가
            edge_xs.extend([bx, x, None])
            edge_ys.extend([by, y, None])
            edge_zs.extend([bz, z, None])
            
            sizes.append(max(8, rel / 4.0)) # 관련성 높으면 마커 크기 크게
            colors.append(rel) # 색상
            texts.append(f"{item.get('file_name','문서')}<br>{rel}%")

        # 3D 그래프의 trace 업데이트
        with self.fig3d.batch_update():
            self.fig3d.data[self.search_trace_idx].x = xs
            self.fig3d.data[self.search_trace_idx].y = ys
            self.fig3d.data[self.search_trace_idx].z = zs
            self.fig3d.data[self.search_trace_idx].marker.size = sizes
            self.fig3d.data[self.search_trace_idx].marker.color = colors
            self.fig3d.data[self.search_trace_idx].text = texts
            
            self.fig3d.data[self.query_edge_trace_idx].x = edge_xs
            self.fig3d.data[self.query_edge_trace_idx].y = edge_ys
            self.fig3d.data[self.query_edge_trace_idx].z = edge_zs
        
        # 막대 차트 업데이트
        file_names_bar = [res['file_name'] for res in res_sorted]
        relevances_bar = [res['relevance'] for res in res_sorted]
        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = file_names_bar
            self.bar_fig.data[0].y = relevances_bar
            self.bar_fig.data[0].text = [f"{r}%" for r in relevances_bar]
            self.bar_fig.data[0].marker.color = relevances_bar
            self.bar_fig.data[0].marker.colorscale = 'Viridis'
            self.bar_fig.data[0].marker.cmin = 0
            self.bar_fig.data[0].marker.cmax = 100
            self.bar_fig.data[0].marker.showscale = True
        
        self.bar_fig.update_layout(
            xaxis_tickangle=-45,
            margin=dict(b=100, t=40)
        )

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수 (시각화 로직에서 에이전트 프로세스 제거)
# ================================================================
# 에이전트 함수들은 LangGraph 로직만 처리하며, 시각화는 필요한 시점에 visualizer를 직접 호출
# (이전 update_agent_status 호출은 삭제되었음)

def extractor_agent(state: AgentState):
    """사용자 쿼리에서 키워드 추출"""
    # 쿼리가 입력되면 3D 시각화 중앙의 텍스트를 업데이트
    visualizer._update_query_text(state["query"])
    
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    formatted_prompt = keyword_prompt.format(query=state["query"])
    keywords = LLM.invoke(formatted_prompt)
    
    return {**state, "keywords": keywords.strip()}

def rag_search_agent(state: AgentState):
    """검색 및 관련성 계산, 3D 시각화 업데이트"""
    try:
        # 1. ChromaDB 로드 및 검색
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results: 
            # 검색 결과가 없으면 3D 시각화에서 문서 노드를 비움
            visualizer.update_search_results([], state["query"])
            return {**state, "search_results": [], "context": ""}
        
        # 2. 문서 중복 제거 (가장 높은 유사도 유지)
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")
            if doc_id:
                content = doc.page_content.strip()
                if content:
                    # score: 낮은 값이 더 유사함을 의미하는 경우가 많으므로 < best[doc_id][0]
                    if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                        best_doc_info[doc_id] = (score, content)
        
        if not best_doc_info: 
            visualizer.update_search_results([], state["query"])
            return {**state, "search_results": [], "context": ""}
        
        # 3. 유사도 정규화 (0-100% 스케일)
        scores = [score for score, _ in best_doc_info.values()]
        min_score, max_score = min(scores), max(scores)
        
        # 4. MySQL에서 파일 메타데이터 조회
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor(pymysql.cursors.DictCursor)
        search_results_with_metadata = []
        
        for doc_id, (score, content) in best_doc_info.items():
            cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
            row = cursor.fetchone()
            if row:
                relevance = (1 - (score - min_score) / (max_score - min_score)) * 100 if max_score != min_score else 100
                search_results_with_metadata.append({
                    "relevance": round(relevance, 1),
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": row["summary"][:100] + "..." if row["summary"] else "요약 없음",
                    "doc_type": row["doc_type"],
                    "content": content
                })
        conn.close()
        
        # 5. 관련성 순으로 정렬 및 컨텍스트 생성
        search_results_with_metadata.sort(key=lambda x: x["relevance"], reverse=True)
        context = "\n\n".join([res['content'] for res in search_results_with_metadata[:10]])
        
        # 6. 3D 시각화 업데이트
        visualizer.update_search_results(search_results_with_metadata, state["query"])
        
        return {**state, "search_results": search_results_with_metadata, "context": context}
    
    except Exception as e:
        print(f"❌ RAG 검색 오류: {e}")
        # 오류 발생 시 3D 시각화에서도 문서 노드를 비움
        visualizer.update_search_results([], state["query"]) 
        return {**state, "search_results": [], "context": ""}

def answer_generator_agent(state: AgentState):
    """검색된 문서를 바탕으로 답변 생성"""
    if not state["search_results"]:
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}
    
    search_summary = "\n".join([
        f"- {res['file_name']} ({res['relevance']}%)" 
        for res in state["search_results"]
    ])
    
    prompt = ChatPromptTemplate.from_template(
        """다음 검색 결과 요약과 문서 내용을 바탕으로 사용자 질문에 답변해 주세요.
        - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
        - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
        - 항상 공손하고 전문적인 어조를 유지해 주세요.

        ### 검색 결과 요약:
        {search_summary}

        ### 문서 내용:
        {context}

        ### 사용자 질문:
        {query}

        ### 답변:
        ==최종결론==
        찾은 문서중에 가장 관련이 높은 파일을 찾아주고 자세한 파일내용 설명해줘
        (순위, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
        """
    )
    chain = prompt | CHAT_LLM | StrOutputParser()
    result = chain.invoke({
        "search_summary": search_summary,
        "context": state["context"],
        "query": state["query"]
    })
    
    return {**state, "result": result.strip()}

def result_formatter_agent(state: AgentState):
    """최종 답변 및 검색 결과 형식화"""
    formatted_result = "🔍 검색된 문서 목록:\n"
    if not state["search_results"]:
        formatted_result += "  - 검색 결과 없음\n"
    else:
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += (
                f"\n--- {i}순위 ({res['relevance']}%) ---\n"
                f"📄 파일명: {res['file_name']}\n"
                f"   📁 위치: {res['file_location']}\n"
                f"   📝 요약: {res['summary']}\n"
                f"   🏷️ 유형: {res['doc_type']}\n"
            )
    
    formatted_result += f"\n💬 AI 답변:\n{state['result']}"
    return {**state, "result": formatted_result}

# ================================================================
# 7. LangGraph 구성 및 실행
# ================================================================
graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)

app = graph.compile()

# ================================================================
# 8. 실행 예시
# ================================================================
visualizer.show()

# RAG 파이프라인 실행
state = {"query": "한문철 TV", "keywords": "", "search_results": [], "context": "", "result": ""}
result = app.invoke(state)

print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

    'data': [{'marker': {'color': 'red', 'line': {'color': 'darkr…


📊 RAG 처리 완료! 최종 결과:
🔍 검색된 문서 목록:

--- 1순위 (100.0%) ---
📄 파일명: IT기사.docx
   📁 위치: fileList/IT기사.docx
   📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐...
   🏷️ 유형: .docx

--- 2순위 (77.7%) ---
📄 파일명: IT과학정보.docx
   📁 위치: fileList/IT과학정보.docx
   📝 요약: SK텔레콤이 SK AX와 협력하여 개발한 AI 업무 지원 시스템 '에이닷 비즈'가 연말까지 SK그룹 내 25개 사에 도입될 예정이다. 이 시스템은 자연어 처리를 통해 회의록 작성 ...
   🏷️ 유형: .docx

--- 3순위 (66.9%) ---
📄 파일명: 속도미달.docx
   📁 위치: fileList/속도미달.docx
   📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된...
   🏷️ 유형: .docx

--- 4순위 (58.0%) ---
📄 파일명: 세계기사.docx
   📁 위치: fileList/세계기사.docx
   📝 요약: 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 과도한 가족 압박과 외로움으로 인해 뉴질랜드 이주 후 다양한 도시에서 생활하며 학문적 성공을 이루었으나, 현재는 한 달에 약 2...
   🏷️ 유형: .docx

--- 5순위 (46.6%) ---
📄 파일명: 해외축구_기사.docx
   📁 위치: fileList/해외축구_기사.docx
   📝 요약: 아틀레티코 마드리드는 잉글랜드 출신 공격수 메이슨 그린우드를 영입하려는 의지를 보이며, 마르세유와는 이적료 약 1225억원으로 협상 중이다. 그린우드는 맨체스터 유나이티드 유스 시